In [24]:
# importing the needed libraries/modules/packages

import math
import numpy as np
import pandas as pd
from sklearn import datasets, model_selection, preprocessing as PP

In [25]:
# loading the breast cancer dataset and getting the input, output and the feature names

data=datasets.load_breast_cancer()

X=data.data
Y=data.target

columns=data.feature_names
N=len(columns)

In [26]:
# describing the input data through a dataframe

df=pd.DataFrame(X, columns=columns)

df.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [27]:
# scaling the data, and we're standardising it due to the presence of outliers as it will make the calculations fast and will remove the dominance of the features with large scale over those with small scale

scaler=PP.StandardScaler()

X=scaler.fit_transform(X)

X=np.concatenate((X, np.ones(X.shape[0]).reshape(-1, 1)), axis=1)

In [28]:
# splitting the data into training and testing, getting numpy arrays for the training part and concatinating a column with all 1s  to the last

X_train, X_test, Y_train, Y_test=model_selection.train_test_split(X, Y, random_state=1)

M=X_train.shape[0]

np_x=np.array(X_train)
np_y=np.array(Y_train)

In [29]:
# score function which predicts the mean average score or the accuracy for the predictions made

def score(Y_pred, Y_true):
    count_right_predictions=0
    total_predictions=len(Y_pred)

    for i in range(len(Y_pred)):        
        if Y_pred[i]==Y_true[i]:
            count_right_predictions+=1

    score=(count_right_predictions/total_predictions)

    return score

In [30]:
# predicting the type of breast cancer as malignant or benign depending on the feature values, using the optimal coefficients calculated through gradient descent 

def predict(x, coeffs):
    y_pred=np.zeros(x.shape[0])

    for i in range(x.shape[0]):
        mtxi=(coeffs*x[i]).sum()
        hxi=1/(1+math.exp(-mtxi))

        y_pred[i]=hxi

    return y_pred

In [31]:
# step gradient function which makes a change in the value of the coefficients, so that we move closer towards the optimal cost 

def step_gradient(coeffs, learning_rate):
    slope=np.zeros(N+1)    

    for j in range(N+1):
        for i in range(M):
            yi=np_y[i]
            mtxi=(coeffs*np_x[i]).sum()
            hxi=1/(1+math.exp(-mtxi))
            xij=np_x[i][j]

            slope[j]+=(xij*(yi-hxi))

    slope/=(-M)
    coeffs-=(learning_rate*slope)

    return coeffs

In [32]:
# cost function which calculates the cost/error for a particular value of the coefficients 

def cost(coeffs):  
    cost=0
     
    for i in range(M):
        yi=np_y[i]        
        mtxi=(coeffs*np_x[i]).sum()

        ei=(yi*mtxi)-math.log(1+math.exp(mtxi))

        cost+=ei

    cost/=(-M)

    return cost

In [33]:
# gd function which performs the gradient descent and returns the optimal coefficients for which the cost is minimised

def gd(learning_rate):    
    coeffs=np.zeros(N+1)

    prev_cost=cost(coeffs)

    # choosing the value of learning rate or aplha for which we just don't overshoot and the cost starts decreasing

    while True:                
        new_coeffs=step_gradient(coeffs, learning_rate)
        new_cost=cost(new_coeffs)    

        if new_cost>=prev_cost:
            learning_rate/=10                 
        else:
            break

    i=0

    while True:   
        prev_cost=cost(coeffs)

        coeffs=step_gradient(coeffs, learning_rate)
        new_cost=cost(coeffs)

        print("Cost", i, new_cost)

        # when the difference between the new and prev cost is <= 0.0001, then we simply break and return the optimal coefficients 

        if abs(new_cost-prev_cost)<=0.0001:
            break

        i+=1

    return coeffs 

In [34]:
# run function which runs the gradient descent code

def run():
    # initial value of learning rate or alpha
    
    learning_rate=0.1

    coeffs=gd(learning_rate)

    return coeffs

In [35]:
# getting the optimal coefficients through the gradient descent algorithm 

coeffs=run()

Cost 0 0.42624090990666574
Cost 1 0.3719832908404567
Cost 2 0.3348206054307278
Cost 3 0.30741806679756645
Cost 4 0.28617363691709097
Cost 5 0.26909556230910586
Cost 6 0.25498572215655624
Cost 7 0.24307646906945968
Cost 8 0.23285118467683255
Cost 9 0.2239481977200138
Cost 10 0.21610596863283316
Cost 11 0.20913016428533712
Cost 12 0.2028730182678479
Cost 13 0.1972199179346669
Cost 14 0.19208041505449802
Cost 15 0.18738203833948452
Cost 16 0.1830659341015939
Cost 17 0.17908373111393008
Cost 18 0.17539524430182865
Cost 19 0.17196676506075712
Cost 20 0.16876976939150398
Cost 21 0.1657799285567015
Cost 22 0.16297634206644387
Cost 23 0.16034093629102103
Cost 24 0.15785798800424253
Cost 25 0.15551374324714137
Cost 26 0.15329610969690893
Cost 27 0.15119440628247777
Cost 28 0.14919915779924076
Cost 29 0.14730192520451685
Cost 30 0.14549516443750313
Cost 31 0.14377210821935796
Cost 32 0.14212666650205352
Cost 33 0.14055334215545798
Cost 34 0.13904715918696456
Cost 35 0.13760360133179012
Cost 36 0

In [36]:
# getting the predictions for the test data and setting them to either 0 or 1

Y_pred=predict(X_test, coeffs)

for i in range(len(Y_pred)):
    if Y_pred[i]<=0.5:
        Y_pred[i]=0
    else:
        Y_pred[i]=1

In [37]:
# storing the predictions in a csv file

df=pd.DataFrame(np.array(Y_pred, dtype=int))

df.to_csv("predictions.csv", header=False, index=False)

In [38]:
# printing the score for the gradient descent algorithm 

score=score(Y_pred, Y_test)

print(score)

0.958041958041958
